# <center> Beach Volleyball -- Team 10

In [1]:
import requests
from bs4 import BeautifulSoup
import openpyxl

In [2]:
# Create an Excel workbook
workbook = openpyxl.Workbook()

# Create a sheet in the workbook
sheet = workbook.active
sheet.title = "Beach_Vital_Statistics"

# Add header row to the sheet
sheet.append(["Player ID", "Attribute", "Value"])

In [38]:
for player_id in range(1, 22105): #1 to 22454
    url = f"http://www.bvbinfo.com/player.asp?ID={player_id}"

    # Make a request to the URL
    try:
        response = requests.get(url, timeout=10)
    except requests.exceptions.Timeout as e:
        # If the request times out, skip to the next player ID
        print(f"Time out for player ID {player_id}: {e}")
        continue
        
    except requests.exceptions.RequestException as e:
        # If there is another error, skip to the next player ID
        print(f"Error for player ID {player_id}: {e}")
        continue
        
    
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Get player name and country
    name_element = soup.find(class_="clsPlayerName")
    if name_element:
        name = name_element.text.strip()
        country_element = soup.find(class_="clsPlayerCountry")
        if country_element:
            country = country_element.text.strip()
            # Put player ID, name and country in the sheet
            sheet.append([player_id, "Name", name])
            sheet.append([player_id, "Country", country])
        else:
            country = ' '
            sheet.append([player_id, "Country", country])
    else:
        name = ' '
        sheet.append([player_id, "Name", name])
    

    # Find the element with the "Vital Statistics" string
    vital_statistics = soup.find(string="Vital Statistics")

    if vital_statistics:
        # Find the parent table of the "Vital Statistics"
        table = vital_statistics.find_parent("table")

        if table:
            # Iterate over the rows in the table
            for row in table.find_all("tr"):
                # Get the columns in the row
                columns = row.find_all("td")

                # If the row has at least 2 columns of data and without the 'Photo' in the first column
                if len(columns) >= 2 and "Photo" not in columns[0].text:
                    
                    # Get the text in the first column (when class="clsPlayerDataLabel")
                    # Let the text in the first column become the key
                    key = columns[0].text.strip()

                    # Get the text in the second column (when class="clsPlayerData")
                    # Let the text in the second column become the value
                    value = columns[1].text.strip()
                    
                    if len(columns) >= 3:
                        # Split the text from the second column and the third column
                        value += ';'
                        
                        # Get the text in the third column (when class="clsPlayerData")
                        value += columns[2].text.strip()
                    
                    # Print the key and value for checking
                   # print(f"{player_id}: {key}: {value}")
                    
                    # Let the player ID, key, and value to the sheet
                    sheet.append([player_id, key, value])

            #print()
                    
# Save the workbook to disk
workbook.save("Beach_Volleyball2.xlsx")